In [1]:
from dataclasses import dataclass, asdict

import torch, wandb
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from utils.config import Config, ModelType
from utils.model.seq2seq import ThinkingRNN, ThinkingLSTM, ThinkingLearnableDelayRNN
from utils.data import SwapDataset, collate_fn
from tqdm.auto import tqdm

# torch.autograd.set_detect_anomaly(True)

config = Config()
config.model_type = ModelType.LEARNABLE_DELAY_RNN
config.input_size = 11
config.num_classes = 10
config.max_delay = 20
config.seq_min = 5         # 최소 시퀀스 길이
config.seq_max = 20        # 최대 시퀀스 길이
config.device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

run = wandb.init(project="QSWAP_RNN", name=f"{config.model_type.name}", config=asdict(config))
run.__enter__()

print(f"Using device: {config.device}")

wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from /root/.netrc.
wandb: Currently logged in as: sizzflair97 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Using device: cuda:1


In [2]:
DATASET_SIZE = 5000 # 한 Epoch에 사용할 데이터 수
train_dataset = SwapDataset(size=DATASET_SIZE, k=config.input_size-1, min_len=config.seq_min, max_len=config.seq_max)
test_dataset = SwapDataset(size=DATASET_SIZE//10, k=config.input_size-1, min_len=config.seq_min, max_len=config.seq_max)

# DataLoader 생성 (collate_fn 등록 필수!)
train_loader = DataLoader(
    train_dataset, 
    batch_size=config.batch_size,       # 확인용으로 작은 배치
    shuffle=True,       # 학습 시 셔플 추천
    collate_fn=collate_fn # 우리가 만든 패딩 함수 적용
)
test_loader = DataLoader(
    test_dataset, 
    batch_size=config.batch_size, 
    shuffle=False, 
    collate_fn=collate_fn
)

In [ ]:
torch.manual_seed(42) # 재현성을 위해 시드 고정

match config.model_type:
    case ModelType.RNN:
        model = ThinkingRNN(config.input_size, config.hidden_size, config.num_classes, config=config).to(config.device)
    case ModelType.LSTM:
        model = ThinkingLSTM(config.input_size, config.hidden_size, config.num_classes, config=config).to(config.device)
    case ModelType.LEARNABLE_DELAY_RNN:
        model = ThinkingLearnableDelayRNN(config.input_size, config.hidden_size, config.num_classes, max_delay=config.max_delay, config=config).to(config.device)
    case _:
        raise ValueError(f"Unknown model type: {config.model_type}")
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

# 5. 학습 루프
for epoch in tqdm(range(config.epochs), desc="Epochs"):
    total_loss = 0
    model.train()
    for i, (inputs, targets, lengths) in tqdm(enumerate(train_loader), total=len(train_loader), desc="Batches", leave=False):
        inputs, targets = inputs.to(config.device), targets.to(config.device)
        
        optimizer.zero_grad()
        
        # 모델 Forward
        outputs = model(inputs)
        
        # Loss
        # outputs: [Batch, Max_Len, K] -> Flatten
        # targets: [Batch, Max_Len] -> Flatten (-1은 ignore_index 처리됨)
        # loss = criterion(outputs.reshape(config.batch_size, config.seq_max, config.num_classes), targets.reshape(-1))
        valid_mask = targets != -1
        outputs_flat = outputs[valid_mask].view(-1, config.num_classes) 
        targets_flat = targets[valid_mask]
        
        
        loss = criterion(outputs_flat, targets_flat)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        total_loss += loss.item()
        
        with torch.no_grad():
            valid_mask = targets_flat != -1
            if valid_mask.sum() > 0:
                predictions = outputs_flat.argmax(dim=-1)
                correct_predictions = (predictions[valid_mask] == targets_flat[valid_mask]).float()
                accuracy = correct_predictions.mean().item()
            else:
                accuracy = 0.0
        
        wandb.log({"Loss/Train": loss.item(),
                "Accuracy/Train": accuracy})
                
        if (i+1) % 300 == 0:
            print(f'Epoch [{epoch+1}/{config.epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}, Acc: {accuracy:.4f}')

    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, targets, lengths in test_loader:
            inputs, targets = inputs.to(config.device), targets.to(config.device)
            
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 2)
            total += targets.size(0)
            correct += torch.all(predicted == targets, dim=1).sum().item()
        
        wandb.log({"Accuracy/Validation": 100 * correct / total})
        print(f'Validation Accuracy after Epoch {epoch+1}: {100 * correct / total:.2f}%')
            
# 6. 평가 루프
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, targets, lengths in tqdm(test_loader, desc="Testing"):
        inputs, targets = inputs.to(config.device), targets.to(config.device)
        
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 2)
        total += targets.size(0)
        correct += torch.all(predicted == targets, dim=1).sum().item()
        wandb.log({"Accuracy/Test": 100 * correct / total})

    print(f'Test Accuracy of the RNN on the 10000 test images (QSWAP): {100 * correct / total:.2f}%')

TypeError: ThinkingLearnableDelayRNN.__init__() got multiple values for argument 'max_delay'

In [ ]:
outputs.shape

torch.Size([64, 20, 10])

In [ ]:
outputs.shape, targets.shape

(torch.Size([64, 23, 10]), torch.Size([64, 20]))

In [ ]:
run.__exit__(None, None, None)